In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go



In [ ]:
# Load data
with open('data-for-publication.xlsx', 'rb') as f:
    dogs = pd.read_excel(f, sheet_name='Dataset', header=0)
display(dogs)

In [ ]:
# for col_name in ['breed_group', 'sex', 'sterilization_status']:
# figure, axes = plt.subplots(1,3, figsize=(20,8))
# sns.barplot(dogs['breed_group'].value_counts().reset_index(), x='breed_group', y='count', ax=axes[0])
fig = px.bar(dogs['breed_group'].value_counts().reset_index(), y='breed_group', x='count', title='Pocty jednotlivych plemien v datach', width=1000, height=600)
fig.show()

In [ ]:
df = dogs.groupby(by=['sterilization_status','sex'])['dog.id'].count()#.rename(columns={'dog.id':'count'})#.reset_index().set_index(['sex', 'sterilization_status'])
display(df)
display(df['sterilized', 'male'])

fig = go.Figure(data=[
    go.Bar(name='sterlized', x=['male', 'female'], y=[df['sterilized', 'male'], df['sterilized', 'female']]),
    go.Bar(name='intact', x=['male', 'female'], y=[df['intact', 'male'], df['intact', 'female']])
])
fig.update_layout(barmode='stack', width=700, title='Number of male and female sterilized/intact dogs')
fig.show()

# fig = px.bar(dogs['sex'].value_counts().reset_index(), y='sex', x='count', title='Pocty jednotlivych psov podla poh', width=1000, height=600)
# fig.show()

In [ ]:
df = dogs.groupby(by=['sex'])['weight_kg'].mean()
display(df)

sns.histplot(data=dogs, x='weight_kg', hue='sex', hue_order={'female':'blue', 'male':'blue'})

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=dogs.query('sex == "male"')['weight_kg'], name='Male', hovertext=dogs.query('sex == "male"')['dog.id']))
fig.add_trace(go.Histogram(x=dogs.query('sex == "female"')['weight_kg'], name='Female',  hovertext=dogs.query('sex == "female"')['dog.id']))

# Overlay both histograms
fig.update_layout(barmode='overlay', xaxis_title='Weight in KG')
# Reduce opacity to see both histograms
fig.update_traces(opacity=0.75)
fig.show()

In [ ]:
categories = ['Noise.sensitivity_age',
'Fearfulness_age',
'Aggression_age',
'Surface_age',
'Separation_age',
'Impulsivity_age',
'Personality_age']

ages_melt = dogs[categories + ['dog.id']].melt(id_vars='dog.id')
display(ages_melt)

sns.kdeplot(data=ages_melt, x='value', hue='variable')


In [ ]:
fig = px.line(ages_melt[ages_melt['dog.id'] < 100], x='variable', y='value', color='dog.id', markers=True)
fig.show()

In [ ]:
ages = dogs[categories + ['dog.id']]
display(ages.describe())

display(dogs.query('Surface_age > 100'))

In [ ]:
display(dogs)

In [ ]:
aggressive_columns = ['Aggressive_dogs_samegender', 'Aggressive_people', 'Aggressive_dogs_oppositegender', 'aggressiveness_dominance_score']

means = dogs.groupby('breed_group')[aggressive_columns].mean().reset_index()
means['Aggressive_columns_sum'] = means[aggressive_columns[0]] + means[aggressive_columns[1]] + means[aggressive_columns[2]]
means.sort_values(by='Aggressive_columns_sum', inplace=True)
fig = px.bar(means, y='breed_group', x=aggressive_columns[:-1])

fig.show()

In [ ]:
means['aggressiveness_dominance_score_positive'] = means['aggressiveness_dominance_score'] - means['aggressiveness_dominance_score'].min()
means2 = means.sort_values(by='aggressiveness_dominance_score_positive')
fig = px.bar(means2, y='breed_group', x='aggressiveness_dominance_score_positive')
fig.show()

# Tendecia u chovatelov

In [ ]:
# A = dogs.groupby(['leashed_unleashed', 'dog_breeder'])['leashed_unleashed'].count().rename('count').reset_index()
# A['count'] = A['count'] / A.groupby('dog_breeder')['count'].transform('sum')
# px.bar(A, x='leashed_unleashed', y='count', color='dog_breeder', barmode='group')

Chovatelia maju svojich psov zvycajne bez voditka, asi si viacej doveruju ? Pricom nechovatelia to maju opacne.

In [ ]:
A = dogs[['dog_breeder', 'leashed_unleashed']].value_counts().reset_index()
A['leashed_unleashed'] = pd.Categorical(A['leashed_unleashed'], ordered=True, categories=['unleashed', 'partly_leashed', 'leashed'])
A = A.sort_values(by='leashed_unleashed')
A['total'] = A.groupby('dog_breeder')['count'].transform('sum')
A['percentage'] = A['count'] / A['total']
fig = px.bar(A, x='leashed_unleashed', y='percentage', color='dog_breeder', barmode='group')
fig.update_layout(
    xaxis_title='',
)
fig.show()
display(A)


In [ ]:
sns.stripplot(dogs, x='dog_breeder', y='stranger_aggression_score',jitter=0.2, hue='dog_breeder', alpha=0.3)

In [ ]:
sns.violinplot(dogs, x='dog_breeder', y='stranger_aggression_score',)

In [ ]:
sns.violinplot(dogs, x='dog_breeder', y='Obedient', hue='dog_breeder')